In [1]:
import os
import csv
import json
import torch
import numpy as np
from argparse import ArgumentParser
import mlearn.modeling.multitask as mtl
from mlearn.utils.metrics import Metrics
from mlearn.data.batching import TorchtextExtractor
from mlearn.data.clean import Cleaner, Preprocessors
from mlearn.utils.train import run_mtl_model, train_mtl_model
from torchtext.data import TabularDataset, Field, LabelField, BucketIterator

In [2]:
# Data inputs and outputs
main = 'davidson'
auxi = ['waseem']
datadir = '../data/json/'
results = '../results/'
save_model = '../results/'

# Cleaning and metrics
cleaners = ['lower', 'username', 'url']
metrics = ['f1-score', 'precision', 'recall', 'accuracy']
display_metric = stop_metric = 'f1-score'
dev_metrics = Metrics(metrics, display_metric, stop_metric)
metrics = Metrics(metrics, display_metric, stop_metric)

# Experiment
experiment = 'word'
tokenizer = 'bpe'
seed = 42

# Modelling
# All models
model = 'lstm'
patience = 1
encoding = 'embedding'
loss = 'nlll'
optimizer = 'adam'
shuffle = True
gpu = False
batch_first = True
clip = 1.0

# LSTM
layers = 1

# CNN
window_sizes = "2,3,4"
filters = 128

# Hyper Parameters
embedding = 64
hidden = ["64,64"]
shared = 64
epochs = 10
batch_size = 64
learning_rate = 0.02
dropout = 0.2
nonlinearity = 'tanh'

# MTL specific
batches_epoch = 50
loss_weights = [1.0, 0.5]

In [3]:
# Initialise random seeds
torch.random.manual_seed(seed)
np.random.seed(seed)

# Set up experiment and cleaner
c = Cleaner(processes = cleaners)
exp = Preprocessors('data/').select_experiment(experiment)
onehot = True if encoding == 'onehot' else False

# Load tokenizers
if tokenizer == 'spacy':
    selected_tok  = c.tokenize
elif tokenizer == 'bpe':
    selected_tok = c.bpe_tokenize
elif tokenizer == 'ekphrasis' and args.experiment == 'word':
    selected_tok = c.ekphrasis_tokenize
    annotate = {'elongated', 'emphasis'}
    flters = [f"<{filtr}>" for filtr in annotate]
    c._load_ekphrasis(annotate, flters)
elif tokenizer == 'ekphrasis' and args.experiment == 'liwc':
    ekphr = c.ekphrasis_tokenize
    annotate = {'elongated', 'emphasis'}
    flters = [f"<{filtr}>" for filtr in annotate]
    c._load_ekphrasis(annotate, flters)

    def liwc_toks(doc):
        tokens = ekphr(doc)
        tokens = exp(tokens)
        return tokens
    selected_tok = liwc_toks
tokenizer = selected_tok

In [4]:
# Set up fields
text = Field(tokenize = tokenizer, lower = True, batch_first = True)
label = LabelField()
fields = {'text': ('text', text), 'label': ('label', label)}  # Because we load from json we just need this.

# Load main task training data
if main == 'davidson':
    train, dev, test = TabularDataset.splits(datadir, train = 'davidson_binary_train.json',
                                             validation = 'davidson_binary_dev.json',
                                             test = 'davidson_binary_test.json',
                                             format = 'json', skip_header = True, fields = fields)
text.build_vocab(train)
label.build_vocab(train)
main = {'train': train, 'dev': dev, 'test': test, 'text': text, 'labels': label, 'name': main}

# Load aux tasks
auxillary = []
for aux in auxi:
    # Set up fields
    text = Field(tokenize = tokenizer, lower = True, batch_first = True)
    label = LabelField()
    fields = {'text': ('text', text), 'label': ('label', label)}  # Because we load from json we just need this.

    if aux == 'davidson':
        train, dev, test = TabularDataset.splits(datadir, train = 'davidson_binary_train.json',
                                                 validation = 'davidson_binary_dev.json',
                                                 test = 'davidson_binary_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'hoover':
        train, dev, test = TabularDataset.splits(datadir, train = 'hoover_train.json',
                                                 validation = 'hoover_dev.json',
                                                 test = 'hoover_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'oraby_factfeel':
        train, dev, test = TabularDataset.splits(datadir, train = 'oraby_fact_feel_train.json',
                                                 validation = 'oraby_fact_feel_dev.json',
                                                 test = 'oraby_fact_feel_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'oraby_sarcasm':
        train, dev, test = TabularDataset.splits(datadir, train = 'oraby_sarcasm_train.json',
                                                 validation = 'oraby_sarcasm_dev.json',
                                                 test = 'oraby_sarcasm_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'waseem':
        train, dev, test = TabularDataset.splits(datadir, train = 'waseem_train.json',
                                                 validation = 'waseem_dev.json',
                                                 test = 'waseem_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    elif aux == 'waseem_hovy':
        train, dev, test = TabularDataset.splits(datadir, train = 'waseem_hovy_train.json',
                                                 validation = 'waseem_hovy_dev.json',
                                                 test = 'waseem_hovy_test.json',
                                                 format = 'json', skip_header = True, fields = fields)
    text.build_vocab(train)
    label.build_vocab(train)
    auxillary.append({'train': train, 'dev': dev, 'test': test, 'text': text, 'labels': label, 'name': aux})
    if len(auxillary) == len(auxi): break

In [5]:
# Hyper parameters
dropout = dropout
nonlinearity = nonlinearity
learning_rate = learning_rate
epochs = epochs
batch_size = batch_size
batch_count = batches_epoch
loss_weights = loss_weights

params = dict(shared_dim = shared,
              batch_first = True,
              hidden_dims = [int(hidden) for hidden in hidden[0].split(',')],
              input_dims = [len(main['text'].vocab.stoi)] + [len(aux['text'].vocab.stoi) for aux in auxillary],
              output_dims = [len(main['labels'].vocab.stoi)] + [len(aux['labels'].vocab.stoi) for aux in auxillary],
              )
print(params)

{'shared_dim': 64, 'batch_first': True, 'hidden_dims': [64, 64], 'input_dims': [18176, 9826], 'output_dims': [2, 4]}


In [6]:
if not onehot:
    params.update({'embedding_dims': embedding})
if model == 'lstm':
    params.update({'no_layers': layers})
    model = mtl.OnehotLSTMClassifier if onehot else mtl.EmbeddingLSTMClassifier
else:
    params.update({'non-linearity': nonlinearity})

    if model == 'cnn':
        params.update({'window_sizes': [int(win) for win in window_sizes[0].split(',')],
                       'num_filters': filters})
        model = mtl.OnehotCNNClassifier if onehot else mtl.EmbeddingCNNClassifier
    elif model == 'mlp':
        model = mtl.OnehotMLPClassifier if onehot else mtl.EmbeddingMLPClassifier

model = model(**params)
print(model)

EmbeddingLSTMClassifier(
  (all_parameters): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 18176x64]
      (1): Parameter containing: [torch.FloatTensor of size 9826x64]
      (2): Parameter containing: [torch.FloatTensor of size 64x64]
      (3): Parameter containing: [torch.FloatTensor of size 64]
      (4): Parameter containing: [torch.FloatTensor of size 256x64]
      (5): Parameter containing: [torch.FloatTensor of size 256x64]
      (6): Parameter containing: [torch.FloatTensor of size 256]
      (7): Parameter containing: [torch.FloatTensor of size 256]
      (8): Parameter containing: [torch.FloatTensor of size 256x64]
      (9): Parameter containing: [torch.FloatTensor of size 256x64]
      (10): Parameter containing: [torch.FloatTensor of size 256]
      (11): Parameter containing: [torch.FloatTensor of size 256]
      (12): Parameter containing: [torch.FloatTensor of size 2x64]
      (13): Parameter containing: [torch.FloatTensor of size 2]
     

In [7]:
# Info about losses: https://bit.ly/3irxvYK
if loss == 'nlll':
    loss = torch.nn.NLLLoss()
elif loss == 'crossentropy':
    loss = torch.nn.CrossEntropyLoss()

# Set optimizer
if optimizer == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), learning_rate)
elif optimizer == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), learning_rate)
elif optimizer == 'asgd':
    optimizer = torch.optim.ASGD(model.parameters(), learning_rate)
elif optimizer == 'adamw':
    optimizer = torch.optim.AdamW(model.parameters(), learning_rate)

In [8]:
# Batch data
batchers = []
test_batchers = []
if not onehot:
    train_buckets = BucketIterator(dataset = main['train'], batch_size = batch_size, sort_key = lambda x: len(x))
    main_train = TorchtextExtractor('text', 'label', main['name'], train_buckets)
    batchers.append(main_train)

    dev_buckets = BucketIterator(dataset = main['dev'], batch_size = 64, sort_key = lambda x: len(x))
    dev = TorchtextExtractor('text', 'label', main['name'], dev_buckets)

    test_buckets = BucketIterator(dataset = main['test'], batch_size = 64, sort_key = lambda x: len(x))
    test = TorchtextExtractor('text', 'label', main['name'], test_buckets)
    test_batchers.append(test)

    for aux in auxillary:
        train_buckets = BucketIterator(dataset = aux['train'], batch_size = batch_size, sort_key = lambda x: len(x))
        train = TorchtextExtractor('text', 'label', aux['name'], train_buckets)
        batchers.append(train)

        test_buckets = BucketIterator(dataset = aux['test'], batch_size = 64, sort_key = lambda x: len(x))
        test = TorchtextExtractor('text', 'label', aux['name'], test_buckets)
        test_batchers.append(test)
else:
    train_buckets = BucketIterator(dataset = main['train'], batch_size = batch_size, sort_key = lambda x: len(x))
    train = TorchtextExtractor('text', 'label', main['name'], train_buckets, len(main['text'].vocab.stoi))
    batchers.append(train)

    dev_buckets = BucketIterator(dataset = main['dev'], batch_size = 64, sort_key = lambda x: len(x))
    dev = TorchtextExtractor('text', 'label', main['name'], dev_buckets, len(main['text'].vocab.stoi))

    test_buckets = BucketIterator(dataset = main['test'], batch_size = 64, sort_key = lambda x: len(x))
    test = TorchtextExtractor('text', 'label', main['name'], test_buckets)
    test_batchers.append(test)

    for aux in auxillary:
        train_buckets = BucketIterator(dataset = aux['train'], batch_size = batch_size, sort_key = lambda x: len(x))
        train = TorchtextExtractor('text', 'label', aux['name'], train_buckets, len(aux['text'].vocab.stoi))
        batchers.append(train)

        test_buckets = BucketIterator(dataset = aux['test'], batch_size = 64, sort_key = lambda x: len(x))
        test = TorchtextExtractor('text', 'label', aux['name'], test_buckets)
        test_batchers.append(test)
next(iter(batchers[0]))

(tensor([[    2, 12800,    11,  ...,     1,     1,     1],
         [   96,   114,     4,  ...,     1,     1,     1],
         [  238,   411,     6,  ...,     1,     1,     1],
         ...,
         [   32,  7108,   127,  ...,     1,     1,     1],
         [   63,  1910,  8765,  ...,     1,     1,     1],
         [   12,     2,    32,  ...,     1,     1,     1]]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
         0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]))

In [19]:
batch_writing = dict(mdl_hdr = ['model', 'scores'], batch_writer = csv.writer(open('test', 'w')),
                     model_hdr = ['name'], main_name = 'davidson', hyper_info = ['embedding_dim'],
                     metric_hdr = ['f1-score'], writer = csv.writer(open('test2', 'w')), data_name = 'davidson')
modelling_vars = dict(model = model, batchers = batchers, optimizer = optimizer, loss = loss,
                      metrics = metrics, batch_size = batch_size, epochs = epochs, clip = clip,
                      early_stopping = patience, save_model = results, dev = dev_buckets, dev_metrics = dev_metrics,
                      dev_task_id = 0, batches_per_epoch = batches_epoch, low = False, shuffle = False, 
                      dataset_weights = None, loss_weights = loss_weights, gpu = False, hyperopt = False)

In [10]:
train_mtl_model(**modelling_vars, **batch_writing)

Batch:  64%|██████▍   | 32/50 [00:06<00:03,  4.80it/s, batch_loss=0.0653, epoch_loss=0.0542, task=0, task_score=0.1111]


Batch: 100%|██████████| 50/50 [00:09<00:00,  5.89it/s, batch_loss=0.0326, epoch_loss=0.0520, task=1, task_score=0.1159]
                                                                                                                       
Batch:  24%|██▍       | 12/50 [00:02<00:07,  4.86it/s, batch_loss=0.0650, epoch_loss=0.0499, task=0, task_score=0.1429]


Batch:  90%|█████████ | 45/50 [00:09<00:01,  4.66it/s, batch_loss=0.0313, epoch_loss=0.0483, task=1, task_score=0.3217]


Batch: 100%|██████████| 50/50 [00:10<00:00,  4.44it/s, batch_loss=0.0650, epoch_loss=0.0483, task=0, task_score=0.1672]
                                                                                                                       
Batch:  50%|█████     | 25/50 [00:05<00:06,  4.01it/s, batch_loss=0.0650, epoch_loss=0.0483, task=0, task_score=0.1429]


Batch: 100%|██████████| 50/50 [00:11<00:00,  4.76it/s, batch_loss=0.0317, epoch_loss=0.0458, task=1, task_score=0.4321]
                                                                                                                       
Batch:  10%|█         | 5/50 [00:01<00:11,  4.01it/s, batch_loss=0.0650, epoch_loss=0.0539, task=0, task_score=0.1552]


...

Batch: 100%|██████████| 50/50 [00:11<00:00,  4.14it/s, batch_loss=0.0649, epoch_loss=0.0486, task=0, task_score=0.5135]
                                                                                                                       
Batch:  44%|████▍     | 22/50 [00:05<00:05,  5.06it/s, batch_loss=0.0320, epoch_loss=0.0430, task=1, task_score=0.2712]


Batch: 100%|██████████| 50/50 [00:11<00:00,  4.27it/s, batch_loss=0.0319, epoch_loss=0.0448, task=1, task_score=0.3052]
                                                                                                                       
Batch:   2%|▏         | 1/50 [00:00<00:08,  5.63it/s, batch_loss=0.0651, epoch_loss=0.0487, task=0, task_score=0.3883]


Batch:  68%|██████▊   | 34/50 [00:08<00:04,  3.99it/s, batch_loss=0.0650, epoch_loss=0.0557, task=0, task_score=0.2632]


Batch: 100%|██████████| 50/50 [00:11<00:00,  4.36it/s, batch_loss=0.0325, epoch_loss=0.0539, task=1, task_score=0.1988]
                                                                                                                       
Evaluating model:  95%|█████████▍| 37/39 [00:01<00:00, 31.01it/s]
                                                                                                              

Early stopping: Terminate
Loading weights from epoch 7


In [11]:
print(dev_metrics.scores)

{'f1-score': [0.1656626310434239, 0.16474607084212997, 0.3628525784808495, 0.2057478353597029, 0.45509745543704383, 0.4970382299397844, 0.5228508795354752, 0.4760422276980435, 0.38235553670499867], 'precision': [0.5017581480139763, 0.5034403374896287, 0.49785796921926306, 0.5047894779895528, 0.5520911969897047, 0.5572674792282613, 0.5826826218550243, 0.550113937807486, 0.47901324015752633], 'recall': [0.5112941971323474, 0.5242604037906881, 0.498619957537155, 0.5108661070782805, 0.5292692792072291, 0.5325882110415087, 0.5479663420732219, 0.5280007574125222, 0.4876298388207742], 'accuracy': [0.18328623334679048, 0.1828825191764231, 0.37706903512313283, 0.21396851029471134, 0.5010092854259185, 0.5785224061364553, 0.6112232539362131, 0.540976988292289, 0.4113847396043601], 'loss': [0.06545948106336844, 0.06538571202567281, 0.06533544969308458, 0.065346574321311, 0.06526330964046474, 0.06538201244694598, 0.06518948852991836, 0.06531957334503802, 0.06541401819206795]}


In [20]:
run_mtl_model(train = True, **modelling_vars, **batch_writing)

Batch:  64%|██████▍   | 32/50 [00:08<00:04,  3.97it/s, batch_loss=0.0650, epoch_loss=0.0490, task=0, task_score=0.5022]


Batch: 100%|██████████| 50/50 [00:12<00:00,  4.31it/s, batch_loss=0.0647, epoch_loss=0.0492, task=0, task_score=0.5622]
                                                                                                                       
Batch:  20%|██        | 10/50 [00:02<00:09,  4.24it/s, batch_loss=0.0322, epoch_loss=0.0475, task=1, task_score=0.2396]


Batch:  86%|████████▌ | 43/50 [00:10<00:01,  3.63it/s, batch_loss=0.0643, epoch_loss=0.0488, task=0, task_score=0.5973]


Batch: 100%|██████████| 50/50 [00:12<00:00,  3.71it/s, batch_loss=0.0324, epoch_loss=0.0491, task=1, task_score=0.2912]
                                                                                                                       
Batch:  42%|████▏     | 21/50 [00:04<00:07,  4.04it/s, batch_loss=0.0320, epoch_loss=0.0481, task=1, task_score=0.2704]


Batch: 100%|██████████| 50/50 [00:11<00:00,  4.04it/s, batch_loss=0.0324, epoch_loss=0.0494, task=1, task_score=0.2565]
                                                                                                                       
Training model:  30%|███       | 3/10 [00:41<01:38, 14.08s/it, dev_loss=0.0651, dev_score=0.5324, loss=0.0324]


...

Batch: 100%|██████████| 50/50 [00:11<00:00,  4.53it/s, batch_loss=0.0653, epoch_loss=0.0492, task=0, task_score=0.4781]
                                                                                                                       
Batch:  22%|██▏       | 11/50 [00:02<00:09,  4.21it/s, batch_loss=0.0321, epoch_loss=0.0402, task=1, task_score=0.2264]


Batch:  88%|████████▊ | 44/50 [00:10<00:01,  4.85it/s, batch_loss=0.0324, epoch_loss=0.0482, task=1, task_score=0.2191]


Batch: 100%|██████████| 50/50 [00:11<00:00,  4.80it/s, batch_loss=0.0316, epoch_loss=0.0486, task=1, task_score=0.2539]
                                                                                                                       
Batch:  46%|████▌     | 23/50 [00:04<00:05,  4.92it/s, batch_loss=0.0321, epoch_loss=0.0465, task=1, task_score=0.2868]


Batch: 100%|██████████| 50/50 [00:10<00:00,  5.25it/s, batch_loss=0.0317, epoch_loss=0.0473, task=1, task_score=0.2693]
                                                                                                                       
Evaluating model: 100%|██████████| 39/39 [00:01<00:00, 34.42it/s]
                                                                                                              

Early stopping: Terminate
Loading weights from epoch 4
